In [11]:
%matplotlib inline
import pandas as pd

import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

In [16]:
df0 = pd.read_csv('heart.csv')

In [17]:
df0.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [18]:
# Seleciona colunas e remove os NaNs
df1 = df0[['Age','Sex','Cholesterol','HeartDisease']].dropna()
df = df1.rename(columns={'Age': 'idade', 'Sex': 'sexo', 'Cholesterol': 'nivel_colesterol', 'HeartDisease': 'doencas_cardiacas' })
df.head(10)

,idade,sexo,nivel_colesterol,doencas_cardiacas
0,40,M,289,0
1,49,F,180,1
2,37,M,283,0
3,48,F,214,1
4,54,M,195,0
5,39,M,339,0
6,45,F,237,0
7,54,M,208,0
8,37,M,207,1
9,48,F,284,0


In [ ]:
# M é masculino e F é feminino, 0 é quem não tem doenças cardíacas e 1 é quem tem